# Local Model

In this tutorial, we'll explore working with local LLMs on your own computer using Spring-AI and Kotlin.
While we've focused on remote providers like OpenAI and Anthropic in previous tutorials,
running models locally offers several distinct advantages.
Let's explore how to set up and use local models with the same Spring-AI abstractions we've been learning throughout this series.

Let's explore how to set up and use local models with the same Spring-AI abstractions we've been learning throughout this series.

In [1]:
@file:DependsOn("org.springframework.ai:spring-ai-ollama-spring-boot-starter:1.0.0-M6")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.18.2")

## Why use local models?

Local models may not match the power of cloud-based alternatives, but they offer compelling benefits:

- **Free to use** — no API costs or rate limits
- **Work offline** — no internet connection required
- **Privacy** — process sensitive data without sending it to third parties
- **Full control** — you determine model settings and deployment

Spring-AI supports local models through Ollama,
which makes running LLMs on your machine remarkably straightforward.

## Setting up Ollama

First,
visit [ollama.com](https://ollama.com/)
and install the application by following the simple instructions for your operating system.

Next,
explore the [Models page](https://ollama.com/search)
to find a model that fits your needs.
You'll see options ranging from small,
fast models to larger, more capable ones.
Consider your hardware specifications when choosing —
larger models require more RAM and GPU resources.

For this tutorial, I've selected `deepseek-r1`,
which demonstrates _"thinking"_ capabilities similar to reasoning modes in commercial models.

Let's download and start the model:

```bash
ollama run deepseek-r1
```

This command will download the `deepseek-r1:7b` model
(if you don't already have it) and start it.
The first run may take some time as it downloads the model files.

## Creating a Spring-AI Client for Local Model

Now that our local model is running, let's set up the Spring-AI client:

In [2]:
import org.springframework.ai.chat.client.ChatClient
import org.springframework.ai.ollama.OllamaChatModel
import org.springframework.ai.ollama.api.OllamaApi
import org.springframework.ai.ollama.api.OllamaModel
import org.springframework.ai.ollama.api.OllamaOptions

val ollamaApi = OllamaApi()

val ollamaOptions = OllamaOptions.builder()
    .model("deepseek-r1")
    .temperature(0.7)
    .build()

val ollamaModel = OllamaChatModel.builder()
    .ollamaApi(ollamaApi)
    .defaultOptions(ollamaOptions)
    .build()

val chatClient = ChatClient.create(ollamaModel)

Notice that we don't need any API keys — the model is already running on our device.
The process for creating a ChatModel is very similar to what we've done with OpenAI,
Anthropic, and other providers.

With everything set up, let's test our model with a simple query:

In [3]:
chatClient.prompt("tell me a joke about Kotlin").call().content()

<think>
Okay, so I need to come up with a joke about Kotlin. Hmm, where do I start? Well, I know that Kotlin is a programming language, and it's often compared to Java because it runs on the Java Virtual Machine (JVM). So maybe I can play around with that.

Let me think about the differences between Java and Kotlin. One of the main selling points of Kotlin is that it's more modern and concise, which makes coding easier and less error-prone. There are also some features like null safety, which was a big issue in Java. Oh, right, in Java you can have nullable types with 'null' being allowed, but that can lead to NullPointerExceptions if not handled properly.

In Kotlin, they introduced the concept of 'nullable' and 'non-nullable' variables using the '?' symbol. So, for example, a variable like String? would be nullable, meaning it could hold a String or null. But if you try to call a method on a null value, it will throw an exception automatically, which is called a NullPointerException 

You'll notice the response includes a new section under the `<think>` tag.
This is the reasoning block where you can see how the model formed its response to your request.

Given the amount of information returned,
the model processed quite quickly — another advantage of local models.

## Streaming Responses with Local Models

Let's see how streaming works with local models,
allowing us to watch the reasoning process in real-time.

First, we'll need to add dependencies for working with `Flow`:

In [4]:
%useLatestDescriptors
%use coroutines
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.10.1")

Now, let's create a more complex prompt and stream the response:

In [5]:
import kotlinx.coroutines.reactive.asFlow

val streamResponse = chatClient
    .prompt(
        """
        Generate a response in the format of a fully valid HTML5 document, answering the following question:

        <Question>
        Create a colorful periodic table of Kotlin operators showing precedence levels.
        Include descriptions with examples of each operator in action.
        </Question>

        <Instructions>
        Your answer must include:
        1. The complete structure of an HTML document with <html>, <head> (including <title>) and <body> tags
        2. Semantic markup (h1, h2, p, ul/ol, etc.)
        3. Use tables or lists where appropriate to structure the data.
        4. Include minimalist CSS styles to improve readability.
        5. The page must be fully valid HTML5.

        Do not add explanations outside the HTML markup. The entire answer must be only in HTML format!
        </Instructions>
        """
    )
    .stream()
    .content()
    .asFlow()

val answer = StringBuilder()
runBlocking {
    streamResponse.collect {
        answer.append(it)
        print(it)
    }
}

<think>
Okay, so I need to create a response that's a fully valid HTML5 document answering the question about generating a periodic table of Kotlin operators with precedence levels, descriptions, and examples.

First, I'll start by outlining the structure. The user specified that it should include an html tag with head and body sections. In the head, there should be a title and some CSS for styling. The body will contain the actual content.

I remember from the previous example that the structure included semantic elements like h1, h2, p, and ul. So I'll follow that pattern. The main topic is the periodic table of Kotlin operators, so an h1 heading makes sense. Then, an h2 for the legend explaining precedence levels.

Next, the body content should have a section dedicated to each operator category. Each category can be a section with an h2 and a list of operators. For each operator, I'll include its symbol, precedence level, description, and an example.

I also need to include CSS styl

The response consists of two parts: the reasoning and the final HTML answer.
Let's extract the HTML part and render it in our notebook:

In [6]:
val html = answer.toString().substringAfterLast("```html").substringBeforeLast("```")

HTML(html)

<!DOCTYPE html>
 
 
 
 
 Periodic Table of Kotlin Operators 
 
 
 
 Kotlin Operators Periodic Table 

 
 Color Coding by Precedence Level: 
 
 Highest 
 High 
 Medium 
 Low 
 
 

 
 Arithmetic Operators 
 
 
 !! 
 !! (Double NOT) - Unary logical negation 
 Example: !!(true) → false 
 
 

 Assignment Operators 
 
 
 = 
 = (Assign) - Assigns a value to a variable 
 Example: var x = 5 
 
 

 Comparison Operators 
 
 
 == 
 == (Equal) - Checks if two values are equal 
 Example: 5 == 5 → true 
 
 

 Logical Operators 
 
 
 && 
 && (And) - Logical AND operation 
 Example: true && false → false 
 
 

 Unary Operators 
 
 
 ! 
 ! (Logical NOT) - Negates a boolean value 
 Example: !true → false 
 
 

 Bitwise Operators 
 
 
 ! 
 ! (Bitwise NOT) - Inverts all bits of the operand 
 Example: !0x80 → 0x7F 
 
 

 Other Operators 
 
 
 @ 
 @ (Delegation) - Used for delegate properties 
 Example: @delegate = someDelegate

Local models offer an excellent alternative when you need privacy, offline capability, or want to avoid API costs.
With Spring-AI, the programming interface remains consistent whether you're using cloud-based or local models,
making it easy to switch between them as your needs change.

You can apply all the techniques from our previous tutorials (prompts, streaming, tools, structured outputs, advisors, RAG) with local models as well —
the same Spring-AI abstractions work across providers.